In [ ]:
import numpy as np
import pickle
import matplotlib.pylab as plt
import nglview
import ase, ase.io
import mdtraj

# Part 1: PIMD

In [ ]:
# Visualise the AIMD trajectory run
pimd = ase.io.read('01-PIMD-ice/ICE-Ih-pos-1-1.xyz',':')
nglview.show_asetraj(pimd)

In [ ]:
# Compute time per step:
energy_pimd = np.loadtxt('01-PIMD-ice/ICE-Ih-energy-1.dat')
np.average(energy_pimd[:,7])

In [ ]:
# Analyse Temperature and potential energy
plt.plot(energy_pimd[:,1], energy_pimd[:,4])

plt.ylabel('Temperature')
plt.xlabel('Simulation time [fs]')
plt.show()

In [ ]:
# TODO Plot for potential energy

# Part 2: PI convergence

In [ ]:
fn_ene = ! ls 02-CONVERGENCE/sim-P*/ICE-Ih-energy-1.dat
fn_gyr = ! ls 02-CONVERGENCE/sim-P*/ICE-Ih-centroid-gyr-1.dat

replica = [int(''.join(x for x in fn.split('/')[1] if x.isdigit())) for fn in fn_ene]
res_ene = [np.loadtxt(fn) for fn in fn_ene]
res_gyr = [np.loadtxt(fn) for fn in fn_gyr]

In [ ]:
# Load results and analyse convergence
plt.plot(replica, [np.average(ene[100:,5]) for ene in res_ene], 'o-')

plt.xscale('log', basex=2)
plt.xlabel('Replica')
plt.ylabel('<E$_{pot}$> / hartree')
plt.show()

In [ ]:
# plot convergence for radius of gyration
plt.plot(replica, [np.average(gyr[100:,::3]) for gyr in res_gyr], 'o-', label='O')
plt.plot(replica, [np.average(gyr[100:,1::3]) for gyr in res_gyr], 'o-', label='H')

plt.legend()
plt.xscale('log', basex=2)
plt.xlabel('Replica')
plt.ylabel(r'<r$_{gyr}$> / $\AA$')
plt.show()

# Part 3: Production run

In [ ]:
# Analyse Temperature and potential energy
energy_c = np.loadtxt('03-PRODUCTION/sim-P01/ICE-Ih-energy-1.dat')
energy_q = np.loadtxt('03-PRODUCTION/sim-P32/ICE-Ih-energy-1.dat')

plt.plot(energy_c[:,1], energy_c[:,4])
plt.plot(energy_q[:,1], energy_q[:,4])

plt.ylabel('Temperature')
plt.xlabel('Simulation time [fs]')
plt.show()

In [ ]:
# TODO plot potential

In [ ]:
def load_trj(fn, top):
    top = mdtraj.load_frame(top, 0)
    unitcell_lengths = top.unitcell_lengths
    unitcell_angles = top.unitcell_angles

    trj = mdtraj.load(fn, top=top)
    len_trj = len(trj)
    trj.unitcell_lengths = unitcell_lengths.repeat(len_trj, axis=0)
    trj.unitcell_angles = unitcell_angles.repeat(len_trj, axis=0)
    return trj

In [ ]:
def compute_gr(trj, t1='O', t2='O', n_bins=200):
    top = trj.topology
    idx_t1 = top.select('name ' + t1)
    idx_t2 = top.select('name ' + t2)
    pairs = trj.topology.select_pairs(idx_t1, idx_t2)
    min_dimension = trj[0].unitcell_lengths.min() / 2
    r, g_r = mdtraj.compute_rdf(trj, pairs, (0, min_dimension), n_bins=n_bins)
    return r*10, g_r

In [ ]:
trj_c = load_trj('03-PRODUCTION/sim-P01/ICE-Ih-pos-1-1.xyz', '01-PIMD-ice/ice-Ih.pdb')
trj_q = load_trj('03-PRODUCTION/sim-P32/ICE-Ih-pos-1-1.xyz','01-PIMD-ice/ice-Ih.pdb')

In [ ]:
r_c, gr_c = compute_gr(trj_c, 'O', 'O')
r_q, gr_q = compute_gr(trj_q, 'O', 'O')

In [ ]:
# Plot gr

plt.plot(r_c, gr_c, label='Classical')
plt.plot(r_q, gr_q, label='Quantum')

plt.legend()
plt.xlim(xmin=2)
plt.ylabel('g(r)')
plt.xlabel('Distance / $\AA$')

plt.show()

In [ ]:
# TODO repeat the same for O-H and H-H